# Utah Transit Agency Example
In this example, we'll predict the energy consumption for some trips operated by the Utah Transit Authority (UTA) in Salt Lake City. This requires specifying the GTFS data we are analyzing, processing it to produce RouteE-Powertrain inputs, and running a RouteE-Powertrain model to produce energy estimates. 

In [1]:
import logging
import multiprocessing as mp
import os
from nrel.routee.transit import (
    build_routee_features_with_osm,
    predict_for_all_trips,
    repo_root
)

# Set up logging: Clear any existing handlers
logging.getLogger().handlers.clear()

# Configure basic logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s - %(message)s"
)

# Suppress GDAL/PROJ warnings, which flood the output when we run gradeit
os.environ["PROJ_DEBUG"] = "0"

In [2]:
# Set inputs
n_proc = mp.cpu_count()
input_directory = repo_root() / "sample-inputs/saltlake"
output_directory = repo_root() / "reports/saltlake"
if not output_directory.exists():
    output_directory.mkdir(parents=True)

# Number of trips to include in analysis. If None, all will be analyzed.
n_trips_incl = 100

## Process GTFS Data into RouteE Inputs
`build_routee_features_with_osm()` analyzes a GTFS feed to prepare input features for energy prediction with RouteE-Powertrain. It performs the following steps:
- Upsamples all shapes so they are suitable for map matching
- Uses NREL's `mappymatch` package to match each shape to a set of OpenStreetMap road links.
- Uses NREL's `gradeit` package to add estimated average grade to each road link. USGS elevation tiles are downloaded and cached if needed.

In [3]:
routee_input_df = build_routee_features_with_osm(
    input_directory=input_directory,
    n_trips=n_trips_incl,
    add_road_grade=True,
    n_processes=n_proc,
)


2025-08-07 17:52:19,379 [INFO] gtfs_feature_processing - Feed contains 12037 trips and 89590 shapes
2025-08-07 17:52:19,383 [INFO] gtfs_feature_processing - Restricted feed to 100 trips and 65 shapes
2025-08-07 17:52:22,756 [INFO] gtfs_feature_processing - Finished upsampling
2025-08-07 17:52:22,757 [INFO] gtfs_feature_processing - Original shapes length: 22503
2025-08-07 17:52:22,762 [INFO] gtfs_feature_processing - Upsampled shapes length: 135682
2025-08-07 17:52:47,285 [INFO] gtfs_feature_processing - Finished map matching
2025-08-07 17:52:51,334 [INFO] gtfs_feature_processing - Finished attaching timestamps
2025-08-07 17:52:51,413 [INFO] /Users/dmccabe/repos/public/routee-transit/nrel/routee/transit/prediction/grade/add_grade.py - Running gradeit on 100 trips with 12 processes.
2025-08-07 17:52:51,599 [INFO] nrel.routee.transit.prediction.grade.download - Downloading 4 USGS tiles at ONE_THIRD_ARC_SECOND resolution.
2025-08-07 17:52:51,599 [INFO] nrel.routee.transit.prediction.grade

The output of `build_routee_features_with_osm()` is a DataFrame where each row represents the traversal of a particular road network edge during a particular bus trip. It includes the features needed to make energy predictions with RouteE, such as the travel time reported by OpenStreetMap (`travel_time_osm`), the distance (`distances_ft`), and the estimated road grade as a decimal value (`grade_dec_unfiltered`/`grade_dec_filtered`, depending on whether filtering is used in `gradeit`). 

In [4]:
routee_input_df.head()

,trip_id,shape_id,road_id,start_lat,start_lon,end_lat,end_lon,geom,start_timestamp,end_timestamp,kilometers,travel_time_osm,elevation_ft,distances_ft,grade_dec_unfiltered,elevation_ft_filtered,grade_dec_filtered
0,5167853,226318,"(83530570, 359389591, 0)",40.741590,-111.903630,40.741580,-111.905040,LINESTRING (-12457052.326531623 4974302.360727...,0 days 08:20:57,0 days 08:21:17,0.122937,9.166777,4833.837549,22865.814380,0.0264,4471.599057,-0.0003
1,5167853,226318,"(83531876, 1520496415, 0)",40.762350,-111.835687,40.760720,-111.838920,LINESTRING (-12449484.927735036 4977350.947742...,0 days 07:57:38,0 days 07:58:33,0.332763,29.774838,4250.910537,14711.975536,-0.0396,4454.066169,-0.0012
2,5167853,226318,"(83541878, 83667927, 0)",40.749790,-111.879719,40.749790,-111.881010,LINESTRING (-12454382.99646189 4975505.2531126...,0 days 08:11:24,0 days 08:11:43,0.117344,10.499674,4835.882867,14776.345617,0.0396,4428.865088,-0.0017
3,5167853,226318,"(83542422, 1529792857, 0)",40.763704,-111.836015,40.763402,-111.835897,LINESTRING (-12449530.924948633 4977551.491883...,0 days 07:57:11,0 days 07:57:17,0.039014,2.493504,4230.105718,22641.273690,-0.0268,4398.017191,-0.0014
4,5167853,226318,"(83543141, 83530570, 0)",40.741590,-111.902648,40.741590,-111.903535,LINESTRING (-12456936.843691872 4974308.546283...,0 days 08:20:43,0 days 08:20:55,0.089572,6.678938,4227.654300,3633.694342,-0.0007,4363.543854,-0.0095


## Predict Energy Consumption with RouteE-Powertrain
We can now make energy predictions with the data in `routee_input_df` and any trained RouteE Powertrain model. We'll use `"Transit_Bus_Battery_Electric"`, included in `nrel.routee.powertrain` 1.3.2, which is trained on real-world energy data from an electric bus in Salt Lake City.

`predict_with_all_trips()` provides a convenient wrapper for making energy consumption predictions given a RouteE model and the input variables necessary to predict with it:

In [5]:
routee_vehicle_model = "Transit_Bus_Battery_Electric"
routee_results = predict_for_all_trips(
    routee_input_df=routee_input_df,
    routee_vehicle_model=routee_vehicle_model,
    n_processes=n_proc,
)


`routee_results` contains link-level energy predictions for each trip.

In [6]:
routee_results.head()

,trip_id,shape_id,road_id,geom,kilometers,travel_time_osm,grade_dec_unfiltered,kWhs
0,5167853,226318,"(83530570, 359389591, 0)",LINESTRING (-12457052.326531623 4974302.360727...,0.122937,9.166777,0.0264,0.260417
1,5167853,226318,"(83531876, 1520496415, 0)",LINESTRING (-12449484.927735036 4977350.947742...,0.332763,29.774838,-0.0396,-0.422870
2,5167853,226318,"(83541878, 83667927, 0)",LINESTRING (-12454382.99646189 4975505.2531126...,0.117344,10.499674,0.0396,0.440398
3,5167853,226318,"(83542422, 1529792857, 0)",LINESTRING (-12449530.924948633 4977551.491883...,0.039014,2.493504,-0.0268,-0.019418
4,5167853,226318,"(83543141, 83530570, 0)",LINESTRING (-12456936.843691872 4974308.546283...,0.089572,6.678938,-0.0007,0.051642


We can aggregate over trip IDs to get the total energy estimated per trip.

In [7]:

energy_by_trip = routee_results.groupby("trip_id").agg(
    {"kilometers": "sum", "kWhs": "sum"}
)

In [8]:
energy_by_trip["miles"] = 0.6213712 * energy_by_trip["kilometers"]
energy_by_trip["kwh_per_mi"] = energy_by_trip["kWhs"] / energy_by_trip["miles"]
energy_by_trip.head(10)

,kilometers,kWhs,miles,kwh_per_mi
trip_id,,,,
5167853,16.807261,13.482244,10.443548,1.290964
5168435,18.889891,16.903480,11.737635,1.440110
5168551,18.889891,16.903480,11.737635,1.440110
5168681,18.709610,13.372084,11.625613,1.150226
5168737,29.216242,20.059818,18.154132,1.104973
5168822,29.216242,20.059818,18.154132,1.104973
5168863,29.298935,17.893148,18.205514,0.982842
5171085,17.925086,11.673345,11.138132,1.048052
5171156,17.925086,11.673345,11.138132,1.048052


In [9]:
energy_by_trip["kwh_per_mi"].describe()

count    100.000000
mean       1.090608
std        0.390613
min        0.537419
25%        0.820192
50%        1.029093
75%        1.248737
max        2.584523
Name: kwh_per_mi, dtype: float64

Note that the predicted energy consumption values are relatively low because the current RouteE Transit pipeline does not account for HVAC loads, which are a major contributor to BEB energy usage.